# NLP # Text classification - Bag of Word - course Lazy - NPL Python

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from sklearn.model_selection import train_test_split

In [3]:
df_copom = pd.read_csv('df_copom_label_en.csv')

In [4]:
# df_ = df_copom[df_copom['type'] == 'statement'].copy()

In [5]:
df_copom.head(1)

,Unnamed: 0,meeting_number,date_x,date_y,selic,decision,decision_txt,type,text,num_words,text_raw,num_words_raw,label_hawk_dove,label_next_meet
0,0,117,2006/03/08,2006/03/08,16.5,-0.75,decrease,statement,"march meeting , banco central brasil 's moneta...",55.0,"In the March Meeting, the Banco Central do Br...",67.0,dovish,decrease


In [6]:
df_copom.shape

(163, 14)

In [7]:
df_copom.isnull().sum()

Unnamed: 0         0
meeting_number     0
date_x             0
date_y             0
selic              0
decision           0
decision_txt       0
type               0
text               0
num_words          0
text_raw           0
num_words_raw      0
label_hawk_dove    2
label_next_meet    2
dtype: int64

In [8]:
df = df_copom.dropna().copy()
df.shape

(161, 14)

In [9]:
df.isnull().sum()

Unnamed: 0         0
meeting_number     0
date_x             0
date_y             0
selic              0
decision           0
decision_txt       0
type               0
text               0
num_words          0
text_raw           0
num_words_raw      0
label_hawk_dove    0
label_next_meet    0
dtype: int64

In [10]:
X = df['text'].copy()
y = df['label_hawk_dove'].copy()

In [11]:
y

0       dovish
1       dovish
2       dovish
3       dovish
4       dovish
        ...   
156    neutral
157    neutral
158    neutral
159    neutral
160    neutral
Name: label_hawk_dove, Length: 161, dtype: object

In [12]:
#Perform train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2015)

In [13]:
# X_train, X_test, y_train, y_test

In [14]:
from __future__ import print_function, division
from builtins import range
# Note: you may need to update your version of future
# sudo pip install -U future


import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier
from gensim.models import KeyedVectors

In [15]:
class GloveVectorizer:
  def __init__(self):
    # load in pre-trained word vectors
    print('Loading in word vectors...')
    word2vec = {}
    embedding = []
    idx2word = []
    with open('glove.6B/glove.6B.300d.txt', encoding='utf-8') as f:
      # is just a space-separated text file in the format:
      # word vec[0] vec[1] vec[2] ...
      for line in f:
        values = line.split()
        word = values[0]
        vec = np.asarray(values[1:], dtype='float32')
        word2vec[word] = vec
        embedding.append(vec)
        idx2word.append(word)
    print('Found %s word vectors.' % len(word2vec))

    # save for later
    self.word2vec = word2vec
    self.embedding = np.array(embedding)
    self.word2idx = {v:k for k,v in enumerate(idx2word)}
    self.V, self.D = self.embedding.shape

  def fit(self, data):
    pass

  def transform(self, data):
    X = np.zeros((len(data), self.D))
    n = 0
    emptycount = 0
    for sentence in data:
      tokens = sentence.lower().split()
      vecs = []
      for word in tokens:
        if word in self.word2vec:
          vec = self.word2vec[word]
          vecs.append(vec)
      if len(vecs) > 0:
        vecs = np.array(vecs)
        X[n] = vecs.mean(axis=0)
      else:
        emptycount += 1
      n += 1
    print("Numer of samples with no words found: %s / %s" % (emptycount, len(data)))
    return X

  def fit_transform(self, data):
    self.fit(data)
    return self.transform(data)

In [16]:
class Word2VecVectorizer:
  def __init__(self):
    print("Loading in word vectors...")
    self.word_vectors = KeyedVectors.load_word2vec_format(
      'GoogleNews-vectors-negative300.bin',
      binary=True
    )
    print("Finished loading in word vectors")

  def fit(self, data):
    pass

  def transform(self, data):
    # determine the dimensionality of vectors
    v = self.word_vectors.get_vector('king')
    self.D = v.shape[0]

    X = np.zeros((len(data), self.D))
    n = 0
    emptycount = 0
    for sentence in data:
      tokens = sentence.split()
      vecs = []
      m = 0
      for word in tokens:
        try:
          # throws KeyError if word not found
          vec = self.word_vectors.get_vector(word)
          vecs.append(vec)
          m += 1
        except KeyError:
          pass
      if len(vecs) > 0:
        vecs = np.array(vecs)
        X[n] = vecs.mean(axis=0)
      else:
        emptycount += 1
      n += 1
    print("Numer of samples with no words found: %s / %s" % (emptycount, len(data)))
    return X


  def fit_transform(self, data):
    self.fit(data)
    return self.transform(data)

In [17]:
vectorizer = GloveVectorizer()

X_train_glo = vectorizer.fit_transform(X_train)
y_train_glo = y_train

X_test_glo = vectorizer.transform(X_test)
y_test_glo = y_test

Loading in word vectors...
Found 400000 word vectors.
Numer of samples with no words found: 0 / 128
Numer of samples with no words found: 0 / 33


In [18]:
# create the model, train it, print scores
model = RandomForestClassifier(n_estimators=200)
model.fit(X_train_glo, y_train_glo)
print("train score:", model.score(X_train_glo, y_train_glo))
print("test score:", model.score(X_test_glo, y_test_glo))

train score: 1.0
test score: 0.7575757575757576


In [19]:
vectorizer = Word2VecVectorizer()

X_train_vec = vectorizer.fit_transform(X_train)
y_train_vec = y_train

X_test_vec = vectorizer.transform(X_test)
y_test_vec = y_test

Loading in word vectors...
Finished loading in word vectors
Numer of samples with no words found: 0 / 128
Numer of samples with no words found: 0 / 33


In [20]:
# create the model, train it, print scores
model = RandomForestClassifier(n_estimators=200)
model.fit(X_train_vec, y_train_vec)
print("train score:", model.score(X_train_vec, y_train_vec))
print("test score:", model.score(X_test_vec, y_test_vec))

train score: 1.0
test score: 0.7575757575757576


In [22]:
from sklearn.svm import LinearSVC

In [23]:
model = LinearSVC()
model.fit(X_train_vec, y_train_vec)
print("train score:", model.score(X_train_vec, y_train_vec))
print("test score:", model.score(X_test_vec, y_test_vec))

train score: 0.796875
test score: 0.7575757575757576
